In [ ]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score
import time

# 数据目录
directory = 'F:/sleep-edf-database-expanded-1.0.0/sleep-edf-database-expanded-1.0.0/filtered_single_channel_data'

# 读取滤波后的数据
W = np.load(os.path.join(directory, 'filtered_W.npy'))
N1 = np.load(os.path.join(directory, 'filtered_N1.npy'))
N2 = np.load(os.path.join(directory, 'filtered_N2.npy'))
N3 = np.load(os.path.join(directory, 'filtered_N3.npy'))
REM = np.load(os.path.join(directory, 'filtered_REM.npy'))

# 打印数据的形状
print(f'W shape: {W.shape}')
print(f'N1 shape: {N1.shape}')
print(f'N2 shape: {N2.shape}')
print(f'N3 shape: {N3.shape}')
print(f'REM shape: {REM.shape}')

# 假设每个样本的长度为 3000
sample_length = 3000

# 提取特征
fea_n1 = N1[:1005, 0, :]  # 确保数据形状为 (1005, sample_length)
fea_n2 = N2[:4811, 0, :]
fea_n3 = N3[:1267, 0, :]
fea_rem = REM[:1902, 0, :]
fea_w = W[:18612, 0, :]

# 设定标签
Label1 = np.ones((1005,))
Label2 = 2 * np.ones((4811,))
Label3 = 3 * np.ones((1267,))
Label4 = 4 * np.ones((1902,))
Label5 = 5 * np.ones((18612,))

# 合并数据和标签
Data = np.concatenate((fea_n1, fea_n2, fea_n3, fea_rem, fea_w), axis=0)
Label = np.concatenate((Label1, Label2, Label3, Label4, Label5), axis=0)

# 确定总样本数
total_samples = len(Label)

# 打乱样本顺序
Nums = np.random.permutation(total_samples)
split_index = int(total_samples * 0.6)  # 将 60% 用于训练，40% 用于测试

train_data = Data[Nums[:split_index]]
test_data = Data[Nums[split_index:]]
train_label = Label[Nums[:split_index]]
test_label = Label[Nums[split_index:]]

# 归一化数据到（0，1）之间
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# 搭建SVM模型和验证分类正确率
start_time = time.time()
model = SVC(C=8, gamma=0.8)
model.fit(train_data, train_label)
ptest_y = model.predict(test_data)

# 计算分类指标
conf_mat = confusion_matrix(test_label, ptest_y)
accuracy = accuracy_score(test_label, ptest_y)
class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

# 打印分类准确率
print(f'Accuracy_N1 = {class_accuracy[0] * 100:.2f}%')
print(f'Accuracy_N2 = {class_accuracy[1] * 100:.2f}%')
print(f'Accuracy_N3 = {class_accuracy[2] * 100:.2f}%')
print(f'Accuracy_REM = {class_accuracy[3] * 100:.2f}%')
print(f'Accuracy_W = {class_accuracy[4] * 100:.2f}%')
print(f'Total Accuracy = {accuracy * 100:.2f}%')

end_time = time.time()
print(f'Time taken: {end_time - start_time:.2f} seconds')


In [ ]:

# 打印每个数组的形状和数据类型
print(f'W shape: {W.shape}, dtype: {W.dtype}')
print(f'N1 shape: {N1.shape}, dtype: {N1.dtype}')
print(f'N2 shape: {N2.shape}, dtype: {N2.dtype}')
print(f'N3 shape: {N3.shape}, dtype: {N3.dtype}')
print(f'REM shape: {REM.shape}, dtype: {REM.dtype}')

In [ ]:

# 创建标签
Label1 = np.ones(N1.shape[0])       # N1
Label2 = 2 * np.ones(N2.shape[0])   # N2
Label3 = 3 * np.ones(N3.shape[0])   # N3
Label4 = 4 * np.ones(REM.shape[0])  # REM
Label5 = 5 * np.ones(W.shape[0])    # W

# 组合数据和标签
Data = np.concatenate([N1, N2, N3, REM, W], axis=0)
Label = np.concatenate([Label1, Label2, Label3, Label4, Label5], axis=0)

# 划分训练集和测试集
train_data, test_data, train_label, test_label = train_test_split(Data, Label, test_size=0.5, random_state=42, stratify=Label)

# 归一化到（0，1）之间
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# 训练SVM模型
tic = time.time()
model = SVC(C=8, gamma=0.8)
model.fit(train_data, train_label)

# 测试集预测
ptest_y = model.predict(test_data)

# 计算分类准确率
accuracy = accuracy_score(test_label, ptest_y)
print(f'Overall Accuracy: {accuracy * 100:.2f}%')

# 计算分类指标
conf_mat = confusion_matrix(test_label, ptest_y)
class_accuracies = conf_mat.diagonal() / conf_mat.sum(axis=1)

print(f'Accuracy_N1 = {class_accuracies[0] * 100:.2f}%')
print(f'Accuracy_N2 = {class_accuracies[1] * 100:.2f}%')
print(f'Accuracy_N3 = {class_accuracies[2] * 100:.2f}%')
print(f'Accuracy_REM = {class_accuracies[3] * 100:.2f}%')
print(f'Accuracy_W = {class_accuracies[4] * 100:.2f}%')

toc = time.time()
print(f'Total time: {toc - tic:.2f} seconds')
